# Predicting sentiment from product reviews


The goal of this first notebook is to explore logistic regression(LinearClassifier) and feature engineering with existing sklearn functions.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Use DataFrames to do some feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.



Let's get started!


In [1]:
import numpy as np
import pandas as pd
import math
import string
import matplotlib.pyplot as plt
%matplotlib inline

# Data preparation

We will use a dataset consisting of baby product reviews on Amazon.com.

In [2]:
import pandas as pd
products = pd.read_csv('amazon_baby.csv')
products

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5
6,A Tale of Baby\'s Days with Peter Rabbit,"Lovely book, it\'s bound tightly so you may no...",4
7,"Baby Tracker&reg; - Daily Childcare Journal, S...",Perfect for new parents. We were able to keep ...,5
8,"Baby Tracker&reg; - Daily Childcare Journal, S...",A friend of mine pinned this product on Pinter...,5
9,"Baby Tracker&reg; - Daily Childcare Journal, S...",This has been an easy way for my nanny to reco...,4


Now, let us see a preview of what the dataset looks like.

In [3]:
products.head(10)

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5
6,A Tale of Baby\'s Days with Peter Rabbit,"Lovely book, it\'s bound tightly so you may no...",4
7,"Baby Tracker&reg; - Daily Childcare Journal, S...",Perfect for new parents. We were able to keep ...,5
8,"Baby Tracker&reg; - Daily Childcare Journal, S...",A friend of mine pinned this product on Pinter...,5
9,"Baby Tracker&reg; - Daily Childcare Journal, S...",This has been an easy way for my nanny to reco...,4


In [4]:
products.count()

name      183213
review    182702
rating    183531
dtype: int64

## Build the word count vector for each review

Let us explore a specific example of a baby product.


In [7]:
products.loc[269]

name      The First Years Massaging Action Teether
review                    A favorite in our house!
rating                                           5
Name: 269, dtype: object

In [8]:
ss = products['review'].tail(2)
print(ss)

183529    I am extremely happy with this product. I have...
183530    I love this product very mush . I have bought ...
Name: review, dtype: object


Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Transform the reviews into word-counts.

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuations.

In [9]:
def remove_punctuation(text):
    import string
    #table = str.maketrans(dict.fromkeys(string.punctuation))
    table = str.maketrans({key: None for key in string.punctuation})

    return str(text).translate(table) 


In [10]:
#review_without_punctuation = remove_punctuation(products['review'])
products['review1'] = products['review'].apply(remove_punctuation)
products['review1'][:10]

0    These flannel wipes are OK but in my opinion n...
1    it came early and was not disappointed i love ...
2    Very soft and comfortable and warmer than it l...
3    This is a product well worth the purchase  I h...
4    All of my kids have cried nonstop when I tried...
5    When the Binky Fairy came to our house we didn...
6    Lovely book its bound tightly so you may not b...
7    Perfect for new parents We were able to keep t...
8    A friend of mine pinned this product on Pinter...
9    This has been an easy way for my nanny to reco...
Name: review1, dtype: object

In [11]:
import nltk 
from nltk.tokenize import word_tokenize


In [12]:
from nltk.tokenize import RegexpTokenizer
#s = "Good muffins cost $3.88\nin New York.  Please buy me\ntwo of them.\n\nThanks."
tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+')
#tokenizer.tokenize( str(a) for a in products['review'])
tokenizer.tokenize( 'its me but it cant ur review')

['its', 'me', 'but', 'it', 'cant', 'ur', 'review']

In [13]:
import nltk 
text = ('my name is srikanth')


In [14]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
#tokenizer to remove unwanted elements from out data like symbols and numbers
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(lowercase=True,stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)
text_counts= cv.fit_transform(products['review1'])

In [15]:
text_counts.shape

(183531, 152675)

In [15]:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer
st_wd = text.ENGLISH_STOP_WORDS
c_vector = CountVectorizer(stop_words = st_wd, min_df=.0001, lowercase=1)
transformed_dataset = c_vector.fit_transform(products['review1'].values)
transformed_dataset

<183531x11054 sparse matrix of type '<class 'numpy.int64'>'
	with 5152149 stored elements in Compressed Sparse Row format>

In [16]:
print(transformed_dataset)


  (0, 4513)	1
  (0, 5711)	1
  (0, 9384)	1
  (0, 5163)	1
  (0, 9287)	1
  (0, 8174)	1
  (0, 9214)	1
  (0, 7947)	1
  (0, 6380)	1
  (0, 6213)	1
  (0, 3662)	1
  (0, 6801)	1
  (0, 3670)	1
  (0, 4519)	1
  (0, 10449)	1
  (0, 7588)	1
  (0, 4672)	1
  (0, 5318)	1
  (0, 9779)	1
  (0, 8920)	1
  (0, 6426)	1
  (0, 5465)	1
  (0, 2037)	2
  (0, 10571)	1
  (0, 6681)	1
  :	:
  (183529, 2119)	1
  (183529, 7347)	1
  (183529, 10711)	1
  (183529, 3284)	2
  (183529, 5613)	2
  (183529, 7439)	2
  (183529, 5717)	1
  (183529, 8711)	1
  (183529, 7781)	1
  (183529, 5318)	1
  (183530, 3626)	1
  (183530, 8506)	1
  (183530, 6309)	1
  (183530, 2934)	1
  (183530, 9741)	1
  (183530, 7494)	1
  (183530, 3267)	1
  (183530, 5333)	1
  (183530, 1682)	2
  (183530, 1377)	1
  (183530, 7713)	1
  (183530, 9507)	1
  (183530, 10912)	1
  (183530, 7439)	1
  (183530, 5755)	1


In [17]:
x_names = c_vector.get_feature_names()
#x_names
text_counts = pd.DataFrame(transformed_dataset.toarray() , columns =x_names)

In [18]:
text_counts

,03,04,05,06,07,08,09,10,100,1000,...,zipping,zippy,zips,zoli,zone,zoo,zoom,zooming,zooms,zooper
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
products['text_counts'] = cv.fit_transform(products['review1'])
print(products['text_counts'])

0           (0, 62888)\t1\n  (0, 78933)\t1\n  (0, 128163...
1           (0, 62888)\t1\n  (0, 78933)\t1\n  (0, 128163...
2           (0, 62888)\t1\n  (0, 78933)\t1\n  (0, 128163...
3           (0, 62888)\t1\n  (0, 78933)\t1\n  (0, 128163...
4           (0, 62888)\t1\n  (0, 78933)\t1\n  (0, 128163...
5           (0, 62888)\t1\n  (0, 78933)\t1\n  (0, 128163...
6           (0, 62888)\t1\n  (0, 78933)\t1\n  (0, 128163...
7           (0, 62888)\t1\n  (0, 78933)\t1\n  (0, 128163...
8           (0, 62888)\t1\n  (0, 78933)\t1\n  (0, 128163...
9           (0, 62888)\t1\n  (0, 78933)\t1\n  (0, 128163...
10          (0, 62888)\t1\n  (0, 78933)\t1\n  (0, 128163...
11          (0, 62888)\t1\n  (0, 78933)\t1\n  (0, 128163...
12          (0, 62888)\t1\n  (0, 78933)\t1\n  (0, 128163...
13          (0, 62888)\t1\n  (0, 78933)\t1\n  (0, 128163...
14          (0, 62888)\t1\n  (0, 78933)\t1\n  (0, 128163...
15          (0, 62888)\t1\n  (0, 78933)\t1\n  (0, 128163...
16          (0, 62888)\t1\n  (0, 78933)\

In [20]:
products['text_counts'][22]

<183531x152675 sparse matrix of type '<class 'numpy.int64'>'
	with 5512117 stored elements in Compressed Sparse Row format>

In [21]:
import nltk 
from nltk.tokenize import word_tokenize

In [22]:
ss= 'my name is srikanth and i am doing machine learning course ,which is related to AI '
wd = word_tokenize(ss)
print(wd)
dd = nltk.FreqDist(wd)
dd

['my', 'name', 'is', 'srikanth', 'and', 'i', 'am', 'doing', 'machine', 'learning', 'course', ',', 'which', 'is', 'related', 'to', 'AI']


FreqDist({'is': 2, 'my': 1, 'name': 1, 'srikanth': 1, 'and': 1, 'i': 1, 'am': 1, 'doing': 1, 'machine': 1, 'learning': 1, ...})

In [23]:
products[:10]

,name,review,rating,review1,text_counts
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3,These flannel wipes are OK but in my opinion n...,"(0, 62888)\t1\n (0, 78933)\t1\n (0, 128163..."
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...,"(0, 62888)\t1\n (0, 78933)\t1\n (0, 128163..."
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...,"(0, 62888)\t1\n (0, 78933)\t1\n (0, 128163..."
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...,"(0, 62888)\t1\n (0, 78933)\t1\n (0, 128163..."
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,All of my kids have cried nonstop when I tried...,"(0, 62888)\t1\n (0, 78933)\t1\n (0, 128163..."
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5,When the Binky Fairy came to our house we didn...,"(0, 62888)\t1\n (0, 78933)\t1\n (0, 128163..."
6,A Tale of Baby\'s Days with Peter Rabbit,"Lovely book, it\'s bound tightly so you may no...",4,Lovely book its bound tightly so you may not b...,"(0, 62888)\t1\n (0, 78933)\t1\n (0, 128163..."
7,"Baby Tracker&reg; - Daily Childcare Journal, S...",Perfect for new parents. We were able to keep ...,5,Perfect for new parents We were able to keep t...,"(0, 62888)\t1\n (0, 78933)\t1\n (0, 128163..."
8,"Baby Tracker&reg; - Daily Childcare Journal, S...",A friend of mine pinned this product on Pinter...,5,A friend of mine pinned this product on Pinter...,"(0, 62888)\t1\n (0, 78933)\t1\n (0, 128163..."
9,"Baby Tracker&reg; - Daily Childcare Journal, S...",This has been an easy way for my nanny to reco...,4,This has been an easy way for my nanny to reco...,"(0, 62888)\t1\n (0, 78933)\t1\n (0, 128163..."


Now, let us explore what the sample example above looks like after these 2 transformations. Here, each entry in the **word_count** column is a dictionary where the key is the word and the value is a count of the number of times the word occurs.

In [24]:
products['text_counts'][269]

<183531x152675 sparse matrix of type '<class 'numpy.int64'>'
	with 5512117 stored elements in Compressed Sparse Row format>

## Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [25]:
products = products[products['rating'] != 3]
len(products)
# products size decreases as it will remove below 4 only 4 and 5 are considered and taken it

166752

Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [26]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
products
#apply(lambda variable: condition) as variable is rating  condition is if rating>3=+1 else -1

C:\Users\Srikanth\AppData\Local\conda\conda\envs\SRI\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,name,review,rating,review1,text_counts,sentiment
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...,"(0, 62888)\t1\n (0, 78933)\t1\n (0, 128163...",1
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...,"(0, 62888)\t1\n (0, 78933)\t1\n (0, 128163...",1
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...,"(0, 62888)\t1\n (0, 78933)\t1\n (0, 128163...",1
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,All of my kids have cried nonstop when I tried...,"(0, 62888)\t1\n (0, 78933)\t1\n (0, 128163...",1
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5,When the Binky Fairy came to our house we didn...,"(0, 62888)\t1\n (0, 78933)\t1\n (0, 128163...",1
6,A Tale of Baby\'s Days with Peter Rabbit,"Lovely book, it\'s bound tightly so you may no...",4,Lovely book its bound tightly so you may not b...,"(0, 62888)\t1\n (0, 78933)\t1\n (0, 128163...",1
7,"Baby Tracker&reg; - Daily Childcare Journal, S...",Perfect for new parents. We were able to keep ...,5,Perfect for new parents We were able to keep t...,"(0, 62888)\t1\n (0, 78933)\t1\n (0, 128163...",1
8,"Baby Tracker&reg; - Daily Childcare Journal, S...",A friend of mine pinned this product on Pinter...,5,A friend of mine pinned this product on Pinter...,"(0, 62888)\t1\n (0, 78933)\t1\n (0, 128163...",1
9,"Baby Tracker&reg; - Daily Childcare Journal, S...",This has been an easy way for my nanny to reco...,4,This has been an easy way for my nanny to reco...,"(0, 62888)\t1\n (0, 78933)\t1\n (0, 128163...",1
10,"Baby Tracker&reg; - Daily Childcare Journal, S...",I love this journal and our nanny uses it ever...,4,I love this journal and our nanny uses it ever...,"(0, 62888)\t1\n (0, 78933)\t1\n (0, 128163...",1


In [31]:
#above written code function same as it so no needed.....

#import numpy as np
#for rating in products['rating'] :
#    if rating >3:
#        products['sentiment']=+1
#    else:
#        products['sentiment']=-1 "
products[:10]

,name,review,rating,review1,text_counts,sentiment
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...,"(0, 62888)\t1\n (0, 78933)\t1\n (0, 128163...",1
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...,"(0, 62888)\t1\n (0, 78933)\t1\n (0, 128163...",1
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...,"(0, 62888)\t1\n (0, 78933)\t1\n (0, 128163...",1
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,All of my kids have cried nonstop when I tried...,"(0, 62888)\t1\n (0, 78933)\t1\n (0, 128163...",1
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5,When the Binky Fairy came to our house we didn...,"(0, 62888)\t1\n (0, 78933)\t1\n (0, 128163...",1
6,A Tale of Baby\'s Days with Peter Rabbit,"Lovely book, it\'s bound tightly so you may no...",4,Lovely book its bound tightly so you may not b...,"(0, 62888)\t1\n (0, 78933)\t1\n (0, 128163...",1
7,"Baby Tracker&reg; - Daily Childcare Journal, S...",Perfect for new parents. We were able to keep ...,5,Perfect for new parents We were able to keep t...,"(0, 62888)\t1\n (0, 78933)\t1\n (0, 128163...",1
8,"Baby Tracker&reg; - Daily Childcare Journal, S...",A friend of mine pinned this product on Pinter...,5,A friend of mine pinned this product on Pinter...,"(0, 62888)\t1\n (0, 78933)\t1\n (0, 128163...",1
9,"Baby Tracker&reg; - Daily Childcare Journal, S...",This has been an easy way for my nanny to reco...,4,This has been an easy way for my nanny to reco...,"(0, 62888)\t1\n (0, 78933)\t1\n (0, 128163...",1
10,"Baby Tracker&reg; - Daily Childcare Journal, S...",I love this journal and our nanny uses it ever...,4,I love this journal and our nanny uses it ever...,"(0, 62888)\t1\n (0, 78933)\t1\n (0, 128163...",1


In [43]:
# here the word split and count words fucntion is written from here only the main process gets

import json
with open('important_words.json', 'r') as f: # Reads the list of most frequent words
    important_words = json.load(f)
important_words = [str(s) for s in important_words]


In [45]:
print(important_words)

['baby', 'one', 'great', 'love', 'use', 'would', 'like', 'easy', 'little', 'seat', 'old', 'well', 'get', 'also', 'really', 'son', 'time', 'bought', 'product', 'good', 'daughter', 'much', 'loves', 'stroller', 'put', 'months', 'car', 'still', 'back', 'used', 'recommend', 'first', 'even', 'perfect', 'nice', 'bag', 'two', 'using', 'got', 'fit', 'around', 'diaper', 'enough', 'month', 'price', 'go', 'could', 'soft', 'since', 'buy', 'room', 'works', 'made', 'child', 'keep', 'size', 'small', 'need', 'year', 'big', 'make', 'take', 'easily', 'think', 'crib', 'clean', 'way', 'quality', 'thing', 'better', 'without', 'set', 'new', 'every', 'cute', 'best', 'bottles', 'work', 'purchased', 'right', 'lot', 'side', 'happy', 'comfortable', 'toy', 'able', 'kids', 'bit', 'night', 'long', 'fits', 'see', 'us', 'another', 'play', 'day', 'money', 'monitor', 'tried', 'thought', 'never', 'item', 'hard', 'plastic', 'however', 'disappointed', 'reviews', 'something', 'going', 'pump', 'bottle', 'cup', 'waste', 'retu

In [47]:
for word in important_words:
    products[word] = products['review1'].apply(lambda s : s.split().count(word))
    #to check how the loop runs print below line
print (products[:10])


C:\Users\Srikanth\AppData\Local\conda\conda\envs\SRI\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


                                                 name  \
1                               Planetwise Wipe Pouch   
2                 Annas Dream Full Quilt with 2 Shams   
3   Stop Pacifier Sucking without tears with Thumb...   
4   Stop Pacifier Sucking without tears with Thumb...   
5   Stop Pacifier Sucking without tears with Thumb...   
6            A Tale of Baby\'s Days with Peter Rabbit   
7   Baby Tracker&reg; - Daily Childcare Journal, S...   
8   Baby Tracker&reg; - Daily Childcare Journal, S...   
9   Baby Tracker&reg; - Daily Childcare Journal, S...   
10  Baby Tracker&reg; - Daily Childcare Journal, S...   

                                               review  rating  \
1   it came early and was not disappointed. i love...       5   
2   Very soft and comfortable and warmer than it l...       5   
3   This is a product well worth the purchase.  I ...       5   
4   All of my kids have cried non-stop when I trie...       5   
5   When the Binky Fairy came to our house, we 

In [48]:
products = products[:40000]
col = products.columns
col

Index(['name', 'review', 'rating', 'review1', 'text_counts', 'sentiment',
       'baby', 'one', 'great', 'love',
       ...
       'seems', 'picture', 'completely', 'wish', 'buying', 'babies', 'won',
       'tub', 'almost', 'either'],
      dtype='object', length=199)

In [53]:
y = products['sentiment']
x = products[col[6:]]
print(x.shape,y.shape)

(40000, 193) (40000,)


Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1).

## Split data into training and test sets

Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result.

In [54]:
x.columns

Index(['baby', 'one', 'great', 'love', 'use', 'would', 'like', 'easy',
       'little', 'seat',
       ...
       'seems', 'picture', 'completely', 'wish', 'buying', 'babies', 'won',
       'tub', 'almost', 'either'],
      dtype='object', length=193)

In [55]:
from sklearn.model_selection import train_test_split
x_train, x_test ,y_train,y_test = train_test_split(x , y , test_size =0.2, random_state=40)
print (x_train.shape ,y_train.shape)
print (x_test.shape ,y_test.shape)

(32000, 193) (32000,)
(8000, 193) (8000,)


In [56]:
print(x_train,y_train)

       baby  one  great  love  use  would  like  easy  little  seat   ...    \
21338     1    0      3     0    0      2     1     0       1     0   ...     
20293     0    0      0     0    0      0     0     0       1     0   ...     
12391     0    0      0     0    0      0     0     0       0     0   ...     
23210     0    0      1     0    1      3     1     0       0     0   ...     
14549     0    1      2     0    0      0     0     0       1     0   ...     
23082     0    1      0     0    0      0     0     0       1     0   ...     
29815     0    0      0     0    0      1     0     0       1     2   ...     
18590     0    0      0     0    0      0     0     0       0     0   ...     
21250     1    0      0     0    0      0     0     0       0     0   ...     
13042     1    0      0     0    1      1     0     1       0     0   ...     
40371     3    3      0     0    1      0     0     0       1     0   ...     
898       0    1      0     0    0      0     0     

# Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data. This model will use the column **word_count** as a feature and the column **sentiment** as the target. We will use `validation_set=None` to obtain same results as everyone else.

**Note:** This line may take 1-2 minutes.

In [57]:
from sklearn.linear_model import LogisticRegression
sentiment_model = LogisticRegression()
sentiment_model.fit(x_train ,y_train)

C:\Users\Srikanth\AppData\Local\conda\conda\envs\SRI\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [58]:
y_pred = sentiment_model.predict(x_test)
print(y_pred)


[ 1  1  1 ...  1  1 -1]


In [60]:
sentiment_model.coef_

array([[ 3.16470959e-02,  5.69830098e-02,  8.53100728e-01,
         1.42344071e+00,  8.80116283e-03, -2.34178915e-01,
         1.32434298e-02,  1.18345160e+00,  5.21912297e-01,
        -1.28544102e-01,  2.30006100e-01,  4.28149528e-01,
        -1.57948176e-01,  7.98434328e-02,  4.24142681e-02,
         6.04547179e-02, -6.22742588e-02, -1.49842787e-01,
        -2.17821031e-01,  1.78965283e-01,  1.51326950e-01,
        -2.57858537e-03,  1.55782023e+00, -9.43701404e-02,
        -6.93096337e-02, -5.68418111e-02,  3.04090072e-01,
         2.58252213e-01, -2.28944348e-01,  2.27456149e-01,
         5.31729580e-01,  6.89938376e-02, -3.64406200e-01,
         1.39605023e+00,  7.08972228e-01, -6.60274148e-02,
        -1.00355100e-01,  7.11635635e-02, -9.22518889e-02,
        -1.37814229e-01,  6.75977158e-02,  1.40942406e-03,
        -1.38356582e-02, -2.85613362e-01,  4.30150816e-01,
         7.49199534e-02, -5.10534837e-02,  3.85757761e-01,
         2.00011506e-01, -2.92643193e-01,  2.23539694e-0

**Aside**. You may get a warning to the effect of "Terminated due to numerical difficulties --- this model may not be ideal". It means that the quality metric (to be covered in Module 3) failed to improve in the last iteration of the run. The difficulty arises as the sentiment model puts too much weight on extremely rare words. A way to rectify this is to apply regularization, to be covered in Module 4. Regularization lessens the effect of extremely rare words. For the purpose of this assignment, however, please proceed with the model above.

In [61]:
from sklearn.metrics import roc_auc_score
train_roc_acc = roc_auc_score(y_train , sentiment_model.predict(x_train))
test_roc_acc = roc_auc_score(y_test , sentiment_model.predict(x_test) )
print('train_roc_acc' , train_roc_acc)
print('test_roc_acc' , test_roc_acc)

train_roc_acc 0.6912739374851982
test_roc_acc 0.6874928042368653


Now that we have fitted the model, we can extract the weights (coefficients) as an DataFrame as follows:

In [62]:
from sklearn.metrics import accuracy_score
train_acc = accuracy_score(y_train ,sentiment_model.predict(x_train))
test_acc = accuracy_score(y_test ,y_pred)
print('train_acc' , train_acc)
print('test_acc', test_acc)

train_acc 0.87025
test_acc 0.86525


In [63]:
weights = sentiment_model.coef_
weights

array([[ 3.16470959e-02,  5.69830098e-02,  8.53100728e-01,
         1.42344071e+00,  8.80116283e-03, -2.34178915e-01,
         1.32434298e-02,  1.18345160e+00,  5.21912297e-01,
        -1.28544102e-01,  2.30006100e-01,  4.28149528e-01,
        -1.57948176e-01,  7.98434328e-02,  4.24142681e-02,
         6.04547179e-02, -6.22742588e-02, -1.49842787e-01,
        -2.17821031e-01,  1.78965283e-01,  1.51326950e-01,
        -2.57858537e-03,  1.55782023e+00, -9.43701404e-02,
        -6.93096337e-02, -5.68418111e-02,  3.04090072e-01,
         2.58252213e-01, -2.28944348e-01,  2.27456149e-01,
         5.31729580e-01,  6.89938376e-02, -3.64406200e-01,
         1.39605023e+00,  7.08972228e-01, -6.60274148e-02,
        -1.00355100e-01,  7.11635635e-02, -9.22518889e-02,
        -1.37814229e-01,  6.75977158e-02,  1.40942406e-03,
        -1.38356582e-02, -2.85613362e-01,  4.30150816e-01,
         7.49199534e-02, -5.10534837e-02,  3.85757761e-01,
         2.00011506e-01, -2.92643193e-01,  2.23539694e-0

In [65]:
weights.shape

(1, 193)

There are a total of `121713`(193) coefficients in the model. Recall from the lecture that positive weights $w_j$ correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. 

Fill in the following block of code to calculate how many *weights* are positive ( >= 0).               
(**Hint**: The `'value'` column in DataFrame *weights* must be positive ( >= 0)).

In [66]:
num_positive_weights = len(weights[weights>0])
num_negative_weights = len(weights[weights<0])

print( "Number of positive weights: %s " % num_positive_weights)
print ("Number of negative weights: %s " % num_negative_weights)

Number of positive weights: 85 
Number of negative weights: 108 


**Quiz Question:** How many weights are >= 0?

## Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.  We refer to this set of 3 examples as the **sample_test_data**.

In [67]:
sample_test_data = y_test[10:13]
print (sample_test_data)
sample_test_data

42520    1
10725    1
20731    1
Name: sentiment, dtype: int64


42520    1
10725    1
20731    1
Name: sentiment, dtype: int64

Let's dig deeper into the first row of the **sample_test_data**. Here's the full review:

That review seems pretty positive.

Now, let's see what the next row of the **sample_test_data** looks like. As we could guess from the sentiment (-1), the review is quite negative.

In [68]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test ,y_pred)

0.86525

We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  We will write some code to obtain the **scores** using skleaaarn. For each row, the **score** (or margin) is a number in the range **[-inf, inf]**.

In [90]:
sample_data = x_train.loc[4]
sample_data = np.array(sample_data).reshape(1,-1)
print(sample_data.shape)

(1, 193)


In [103]:
scores = sentiment_model.predict_proba(sample_data)
print (scores)

[[0.06446193 0.93553807]]


In [105]:
sample_test = x_test[10:13]

scores_1 = sentiment_model.predict_proba(sample_test)
print (scores_1)

[[0.13673623 0.86326377]
 [0.16620874 0.83379126]
 [0.3990294  0.6009706 ]]


### Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions:

In [106]:
predictions=[1 for i in scores_1 if i[1]>0.6] # it has to be for probabiltity scores which we get on next time , change i>3 
print (predictions)

[1, 1, 1]


Run the following code to verify that the class predictions obtained by your calculations are the same as that obtained from GraphLab Create.

In [107]:
print ("Class predictions according to sklearn:" )
print (sentiment_model.predict_proba(sample_test) )

Class predictions according to sklearn:
[[0.13673623 0.86326377]
 [0.16620874 0.83379126]
 [0.3990294  0.6009706 ]]


**Checkpoint**: Make sure your class predictions match with the one obtained from sklearn output.

### Probability predictions

Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

In [57]:
import numpy as np
probabilities=1/(1+np.exp(-scores))
print probabilities

[  9.98812385e-01   3.22326818e-03   4.26155800e-07]


**Checkpoint**: Make sure your probability predictions match the ones obtained from GraphLab Create.

In [58]:
print "Class predictions according to GraphLab Create:" 
print sentiment_model.predict(sample_test_data, output_type='probability')

Class predictions according to GraphLab Create:
[0.9988123848377202, 0.003223268181799678, 4.2615579966527684e-07]


** Quiz Question:** Of the three data points in **sample_test_data**, which one (first, second, or third) has the **lowest probability** of being classified as a positive review?

# Find the most positive (and negative) review

We now turn to examining the full test dataset, **test_data**, and use GraphLab Create to form predictions on all of the test data points for faster performance.

Using the `sentiment_model`, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:
1.  Make probability predictions on **test_data** using the `sentiment_model`. (**Hint:** When you call `.predict` to make predictions on the test data, use option `output_type='probability'` to output the probability rather than just the most likely class.)
2.  Sort the data according to those predictions and pick the top 20. (**Hint:** You can use the `.topk` method on an SFrame to find the top k rows sorted according to the value of a specified column.)

In [59]:
test_data['probability_predictions'] = sentiment_model.predict(test_data, output_type='probability')
topk = test_data.topk('probability_predictions', 20)
topk.print_rows(num_rows=20)
#topk() method  is used to get top k rows of predicitons

+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
| Britax Decathlon Convertib... | I researched a few differe... |  4.0   |
| Ameda Purely Yours Breast ... | As with many new moms, I c... |  4.0   |
| Traveling Toddler Car Seat... | I am sure this product wor... |  2.0   |
| Shermag Glider Rocker Comb... | After searching in stores ... |  4.0   |
| Cloud b Sound Machine Soot... | First off, I love plush sh... |  5.0   |
| JP Lizzy Chocolate Ice Cla... | I got this bag as a presen... |  4.0   |
| Fisher-Price Rainforest Me... | My daughter wasn't able to... |  5.0   |
| Lilly Gold Sit 'n' Stroll ... | I just completed a two-mon... |  5.0   |
|  Fisher-Price Deluxe Jumperoo | I had already decided that... |  5.0   |
| North States Supergate Pre... | I got a couple of these to... |  4.0   |
|   Munchkin Mozart Magic

**Quiz Question**: Which of the following products are represented in the 20 most positive reviews? [multiple choice]


Now, let us repeat this exercise to find the "most negative reviews." Use the prediction probabilities to find the  20 reviews in the **test_data** with the **lowest probability** of being classified as a **positive review**. Repeat the same steps above but make sure you **sort in the opposite order**.

In [60]:
bottomk = test_data.sort('probability_predictions')
bottomk.print_rows(num_rows=20)

+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
| Jolly Jumper Arctic Sneak ... | I am a "research-aholic" i... |  5.0   |
| Levana Safe N'See Digital ... | This is the first review I... |  1.0   |
| Snuza Portable Baby Moveme... | I would have given the pro... |  1.0   |
| Fisher-Price Ocean Wonders... | We have not had ANY luck w... |  2.0   |
| VTech Communications Safe ... | This is my second video mo... |  1.0   |
| Safety 1st High-Def Digita... | We bought this baby monito... |  1.0   |
| Chicco Cortina KeyFit 30 T... | My wife and I have used th... |  1.0   |
| Prince Lionheart Warmies W... | *****IMPORTANT UPDATE*****... |  1.0   |
| Valco Baby Tri-mode Twin S... | I give one star to the dim... |  1.0   |
| Adiri BPA Free Natural Nur... | I will try to write an obj... |  2.0   |
| Munchkin Nursery Projec

**Quiz Question**: Which of the following products are represented in the 20 most negative reviews?  [multiple choice]

## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifier. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

Complete the function below to compute the classification accuracy:

In [61]:
def get_classification_accuracy(model, data, true_labels):
    # First get the predictions
    ## YOUR CODE HERE
    predictions=model.predict(data)
    
    # Compute the number of correctly classified examples
    ## YOUR CODE HERE
    correct=np.equal(predictions, true_labels)

    # Then compute accuracy by dividing num_correct by total number of examples
    ## YOUR CODE HERE
    accuracy=sum(correct)/len(data)
    
    return accuracy

Now, let's compute the classification accuracy of the **sentiment_model** on the **test_data**.

In [62]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

0.91453683705303579

**Quiz Question**: What is the accuracy of the **sentiment_model** on the **test_data**? Round your answer to 2 decimal places (e.g. 0.76).

**Quiz Question**: Does a higher accuracy value on the **training_data** always imply that the classifier is better?

## Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subset of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [63]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [64]:
len(significant_words)

20

For each review, we will use the **word_count** column and trim out all words that are **not** in the **significant_words** list above. We will use the [SArray dictionary trim by keys functionality]( https://dato.com/products/create/docs/generated/graphlab.SArray.dict_trim_by_keys.html). Note that we are performing this on both the training and test set.

In [66]:
# here data is trimmed words which are in both sets are taken count reset removed
train_data['word_count_subset'] = train_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)
test_data['word_count_subset'] = test_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)

Let's see what the first example of the dataset looks like:

In [67]:
train_data[0]['review']

'it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.'

The **word_count** column had been working with before looks like the following:

In [68]:
print train_data[0]['word_count']

{'and': 3L, 'love': 1L, 'it': 3L, 'highly': 1L, 'osocozy': 1L, 'bags': 1L, 'leak': 1L, 'moist': 1L, 'does': 1L, 'recommend': 1L, 'was': 1L, 'wipes': 1L, 'disappointed': 1L, 'early': 1L, 'not': 2L, 'now': 1L, 'holder': 1L, 'wipe': 1L, 'keps': 1L, 'wise': 1L, 'i': 1L, 'planet': 1L, 'my': 2L, 'came': 1L}


Since we are only working with a subset of these words, the column **word_count_subset** is a subset of the above dictionary. In this example, only 2 `significant words` are present in this review.

In [69]:
print train_data[0]['word_count_subset']

{'love': 1L, 'disappointed': 1L}


## Train a logistic regression model on a subset of data

We will now build a classifier with **word_count_subset** as the feature and **sentiment** as the target. 

In [70]:
simple_model = graphlab.logistic_classifier.create(train_data,
                                                   target = 'sentiment',
                                                   features=['word_count_subset'],
                                                   validation_set=None)
simple_model

Logistic regression:

--------------------------------------------------------

Number of examples          : 133416

Number of classes           : 2

Number of feature columns   : 1

...

| 5         | 6        | 1.315000     | 0.866815          |

| 6         | 7        | 1.556000     | 0.866815          |

+-----------+----------+--------------+-------------------+

SUCCESS: Optimal solution found.

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 21
Number of examples             : 133416
Number of classes              : 2
Number of feature columns      : 1
Number of unpacked features    : 20

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : newton
Solver iterations              : 6
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 1.616

Settings
--------
Log-likelihood                 : 44323.7254

Highest Positive Coefficients
-----------------------------
word_count_subset[loves]       : 1.6773
word_count_subset[perfect]     : 1.5145
word_count_subset[love]        : 1.3654
(intercept)                    : 1.2995
word_count_subset[easy]        : 1.1937

Lowest Negative Coefficients
----------------------------
word_count_subset[disappointed] : -2.3551
wor

We can compute the classification accuracy using the `get_classification_accuracy` function you implemented earlier.

In [71]:
get_classification_accuracy(simple_model, test_data, test_data['sentiment'])

0.8693004559635229

Now, we will inspect the weights (coefficients) of the **simple_model**:

In [72]:
simple_model.coefficients

name,index,class,value,stderr
(intercept),None,1,1.2995449552,0.0120888541331
word_count_subset,disappointed,1,-2.35509250061,0.0504149888557
word_count_subset,love,1,1.36543549368,0.0303546295109
word_count_subset,well,1,0.504256746398,0.021381300631
word_count_subset,product,1,-0.320555492996,0.0154311321362
word_count_subset,loves,1,1.67727145556,0.0482328275384
word_count_subset,little,1,0.520628636025,0.0214691475665
word_count_subset,work,1,-0.621700012425,0.0230330597946
word_count_subset,easy,1,1.19366189833,0.029288869202
word_count_subset,great,1,0.94469126948,0.0209509926591


Let's sort the coefficients (in descending order) by the **value** to obtain the coefficients with the most positive effect on the sentiment.

In [73]:
simple_model.coefficients.sort('value', ascending=False).print_rows(num_rows=21)

+-------------------+--------------+-------+-----------------+-----------------+
|        name       |    index     | class |      value      |      stderr     |
+-------------------+--------------+-------+-----------------+-----------------+
| word_count_subset |    loves     |   1   |  1.67727145556  | 0.0482328275384 |
| word_count_subset |   perfect    |   1   |  1.51448626703  |  0.049861952294 |
| word_count_subset |     love     |   1   |  1.36543549368  | 0.0303546295109 |
|    (intercept)    |     None     |   1   |   1.2995449552  | 0.0120888541331 |
| word_count_subset |     easy     |   1   |  1.19366189833  |  0.029288869202 |
| word_count_subset |    great     |   1   |  0.94469126948  | 0.0209509926591 |
| word_count_subset |    little    |   1   |  0.520628636025 | 0.0214691475665 |
| word_count_subset |     well     |   1   |  0.504256746398 |  0.021381300631 |
| word_count_subset |     able     |   1   |  0.191438302295 | 0.0337581955697 |
| word_count_subset |     ol

**Quiz Question**: Consider the coefficients of **simple_model**. There should be 21 of them, an intercept term + one for each word in **significant_words**. How many of the 20 coefficients (corresponding to the 20 **significant_words** and *excluding the intercept term*) are positive for the `simple_model`?

In [78]:
10
simple_positive = simple_model.coefficients.sort('value', ascending=False)
simple_positive = simple_positive[simple_positive['value'] > 0]
simple_positive
#print len(simple_positive)

name,index,class,value,stderr
word_count_subset,loves,1,1.67727145556,0.0482328275384
word_count_subset,perfect,1,1.51448626703,0.049861952294
word_count_subset,love,1,1.36543549368,0.0303546295109
(intercept),None,1,1.2995449552,0.0120888541331
word_count_subset,easy,1,1.19366189833,0.029288869202
word_count_subset,great,1,0.94469126948,0.0209509926591
word_count_subset,little,1,0.520628636025,0.0214691475665
word_count_subset,well,1,0.504256746398,0.021381300631
word_count_subset,able,1,0.191438302295,0.0337581955697
word_count_subset,old,1,0.0853961886678,0.0200863423025


**Quiz Question**: Are the positive words in the **simple_model** (let us call them `positive_significant_words`) also positive words in the **sentiment_model**?

In [81]:
# here sentiment rows which are values are > 0
sentiment_positive = sentiment_model.coefficients.sort('value', ascending=False)
sentiment_positive = sentiment_positive[sentiment_positive['value'] > 0]
#sentiment_positive
print len(sentiment_positive)

68419


In [87]:
simple_positive = simple_model.coefficients.sort('value', ascending=False)
simple_positive = simple_positive[simple_positive['value'] > 0]
sentiment_positive = sentiment_model.coefficients.sort('value', ascending=False)
sentiment_positive = sentiment_positive[sentiment_positive['value'] > 0]
print len(simple_positive)
print len(sentiment_positive)
simple_positive = simple_positive.to_numpy()
sentiment_positive = sentiment_positive.to_numpy()
print np.intersect1d(simple_positive, sentiment_positive)
# above line gets words which are in both data 

11
68419
[None 1L '(intercept)' 'able' 'car' 'easy' 'great' 'little' 'love' 'loves'
 'old' 'perfect' 'well']


# Comparing models

We will now compare the accuracy of the **sentiment_model** and the **simple_model** using the `get_classification_accuracy` method you implemented above.

First, compute the classification accuracy of the **sentiment_model** on the **train_data**:

In [88]:
get_classification_accuracy(sentiment_model, train_data, train_data['sentiment'])


0.97944024704683097

Now, compute the classification accuracy of the **simple_model** on the **train_data**:

In [89]:
get_classification_accuracy(simple_model, train_data, train_data['sentiment'])


0.86681507465371466

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TRAINING set?

Now, we will repeat this exercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

In [90]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])


0.91453683705303579

Next, we will compute the classification accuracy of the **simple_model** on the **test_data**:

In [91]:
get_classification_accuracy(simple_model, test_data, test_data['sentiment'])


0.8693004559635229

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TEST set?

## Baseline: Majority class prediction

It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

What is the majority class in the **train_data**?

In [92]:
num_positive  = (train_data['sentiment'] == +1).sum()
num_negative = (train_data['sentiment'] == -1).sum()
print num_positive
print num_negative

112164
21252


Now compute the accuracy of the majority class classifier on **test_data**.

**Quiz Question**: Enter the accuracy of the majority class classifier model on the **test_data**. Round your answer to two decimal places (e.g. 0.76).

In [94]:
pn  = (test_data['sentiment'] == +1).sum()
nn = (test_data['sentiment'] == -1).sum()
print pn
print nn
print pn/ (pn + nn)
# accuracy of pst/sum (pos+neg)

28095
5241
0.842782577394


**Quiz Question**: Is the **sentiment_model** definitely better than the majority class classifier (the baseline)?